# Node

In [1]:
# thanks to: 
# https://www.python-course.eu/python3_properties.php
# http://blog.lerner.co.il/python-attributes/
# https://www.programiz.com/python-programming/property

# this node can store any data types including itself.
class BaseNode(object): 
    def __init__(self, val=object(), max_child_count=0):
        self._dtype = type(val)
        self.value = val
        self._children = [None] * max_child_count
        self._parent = None
            
    @property
    def value(self):        
        return self._data
    
    @value.setter
    def value(self, val):
        self._data = val
        self._dtype = type(val)
        
    def child_count(self):
        return len(self._children) - self._children.count(None)
    
    def max_child_count(self):
        return len(self._children)
        
    def get_child(self, child_index):
        assert 0 <= child_index < len(self._children), 'Invalid child index'
        return self._children[child_index]
    
    def __getitem__(self, cindex):
        return self.get_child(cindex)
    
    def __setitem__(self, cindex, child):
        if child is None:
            self._children[cindex] = None
            return
        assert isinstance(child, type(self)), 'Children must be the same subclass as parent'
        assert 0 <= cindex < len(self._children), 'Invalid child index'                
        if self._children[cindex] is not None:
            # setting child's parent as None
            self._children[cindex]._parent = None        
        self._children[cindex] = child
        child._parent = self
    
    def __repr__(self):
        return str({'child_count':self.child_count(), 'max_children':self.max_child_count(), 
                'value':self._data, 'dtype':self._dtype})
    
    def __str__(self):
        children_str = [str(c.value) if c is not None else '_' for c in self._children]        
        return '({}; {})'.format(self.value, ', '.join(children_str))


class Node(BaseNode):
    @property
    def value(self):        
        return self._data
    
    @value.setter
    def value(self, val):        
        if not isinstance(val, self._dtype):
            raise TypeError('{} data-type cannot be stored in this Node({})'.format(type(val), self._dtype))
        self._data = val
        
    def __setitem__(self, cindex, child):
        if child is None:
            self._children[cindex] = None
            return
        assert isinstance(child, type(self)), 'Children must be the same subclass as parent'
        assert 0 <= cindex < len(self._children), 'Invalid child index'
        assert issubclass(child._dtype, self._dtype), 'dtypes do not match! (%s vs. %s)'\
                                                % (child._dtype, self._dtype)
        assert child.max_child_count() == self.max_child_count(), 'Maximum children must be %d'% self.max_child_count()
        if self._children[cindex] is not None:
            # setting child's parent as None            
            self._children[cindex]._parent = None        
        self._children[cindex] = child
        child._parent = self

In [2]:
print(BaseNode(BaseNode(), 0))
print(repr(BaseNode(BaseNode(), 0)))
print('*'*30)

n = BaseNode(object(),2)
n.value = None

n[0] = BaseNode('l',3)
n[0][0] = Node('ll')
n[0][1] = Node(777, 1)
n[0][2] = Node('lr')

n[1] = Node('r',2)
n[1][1] = Node('rr', 2)

print(n[1])
print(repr(n[1]))
print(n[1][1])
print(repr(n[1][1]))
print('*'*30)
print(n[0])
print(repr(n[0]))
print(n[0][0])
print(repr(n[0][0]))
print(n[0][1])
print(repr(n[0][1]))
print('*'*30)
print(n)
print(repr(n))
print('*'*30)

((<object object at 0x7ff18a7a4d20>; ); )
{'child_count': 0, 'max_children': 0, 'value': {'child_count': 0, 'max_children': 0, 'value': <object object at 0x7ff18a7a4d20>, 'dtype': <class 'object'>}, 'dtype': <class '__main__.BaseNode'>}
******************************
(r; _, rr)
{'child_count': 1, 'max_children': 2, 'value': 'r', 'dtype': <class 'str'>}
(rr; _, _)
{'child_count': 0, 'max_children': 2, 'value': 'rr', 'dtype': <class 'str'>}
******************************
(l; ll, 777, lr)
{'child_count': 3, 'max_children': 3, 'value': 'l', 'dtype': <class 'str'>}
(ll; )
{'child_count': 0, 'max_children': 0, 'value': 'll', 'dtype': <class 'str'>}
(777; _)
{'child_count': 0, 'max_children': 1, 'value': 777, 'dtype': <class 'int'>}
******************************
(None; l, r)
{'child_count': 2, 'max_children': 2, 'value': None, 'dtype': <class 'NoneType'>}
******************************


# Tree

In [3]:
class BinaryTree(Node):
    def __init__(self, val):                
        Node.__init__(self, val, max_child_count=2)
                
    def __repr__(self):
        rep = {}        
        def subtree_repr(node, rep, level):
            if level not in rep.keys():
                rep[level] = list()
            if node is None:                
                rep[level].append('null')
                return
            
            rep[level].append(Node.__repr__(node))            
            for c in node._children:                
                subtree_repr(c, rep, level+1)
        subtree_repr(self, rep, 0)
        return 'tree: ' + repr(rep)
    
    def __str__(self):        
        def subtree_str(node):                                
            if node is None:                
                return '_'
            
            val_str = '(' + str(node.value) + '; '            
            for c in node._children:
                val_str += subtree_str(c) + ', '
            val_str = val_str[:-2] + ')'
            return val_str
        return subtree_str(self)

In [4]:
# example
BT = BinaryTree
root = BT('ROOT')
root[0] = BT('l')
root[1] = BT('r')
root[0][0] = BT('ll')
root[0][1] = BT('lr')
root[1][1] = BT('rr')
root[1][1][1] = BT('rrr')
print('*'*30)
print(str(root) + '\n')
print(repr(root))
print('*'*30)
print(root[1])
print(repr(root[1]))

******************************
(ROOT; (l; (ll; _, _), (lr; _, _)), (r; _, (rr; _, (rrr; _, _))))

tree: {0: ["{'child_count': 2, 'max_children': 2, 'value': 'ROOT', 'dtype': <class 'str'>}"], 1: ["{'child_count': 2, 'max_children': 2, 'value': 'l', 'dtype': <class 'str'>}", "{'child_count': 1, 'max_children': 2, 'value': 'r', 'dtype': <class 'str'>}"], 2: ["{'child_count': 0, 'max_children': 2, 'value': 'll', 'dtype': <class 'str'>}", "{'child_count': 0, 'max_children': 2, 'value': 'lr', 'dtype': <class 'str'>}", 'null', "{'child_count': 1, 'max_children': 2, 'value': 'rr', 'dtype': <class 'str'>}"], 3: ['null', 'null', 'null', 'null', 'null', "{'child_count': 0, 'max_children': 2, 'value': 'rrr', 'dtype': <class 'str'>}"], 4: ['null', 'null']}
******************************
(r; _, (rr; _, (rrr; _, _)))
tree: {0: ["{'child_count': 1, 'max_children': 2, 'value': 'r', 'dtype': <class 'str'>}"], 1: ['null', "{'child_count': 1, 'max_children': 2, 'value': 'rr', 'dtype': <class 'str'>}"], 2

# LinkedList

In [5]:
class LinkedList:
    def __init__(self, fixed_dtype=True):        
        self._head = None        
        self._size = 0
        self.__NodeClass = Node if fixed_dtype else BaseNode
    
    def __len__(self):
        return self._size    
    
    def __getitem__(self, index):
        if not (isinstance(index, int) and 0 <= index < self._size):
            raise IndexError('Invalid index')
        n = self._head
        for i in range(index):
            n = n[0]
        return n.value
        
        
    def __setitem__(self, index, value):
        if not (isinstance(index, int) and 0 <= index < self._size):
            raise IndexError('Invalid index')
        n = self._head
        for i in range(index):
            n = n[0]
        n.value = value
    
    def append(self, value):
        self._size += 1
        if self._head is None:
            self._head = self.__NodeClass(value, 1)
            return
        fw = self._head
        while fw[0] is not None:
            fw = fw[0]
        fw[0] = self.__NodeClass(value, 1)        
        
    def pop(self):
        assert self._head is not None, 'Popping from empty list'
        
        self._size -= 1        
        if self._head[0] is None:
            value = self._head.value
            self._head = None
            return value
        
        fw = self._head
        while fw[0][0] is not None:
            fw = fw[0]
        value = fw[0].value
        fw[0] = None
        return value
    
    def __repr__(self):
        rep = {}
        def list_repr(node, rep, level):
            if node is None:                                
                return
            rep[level] = node
            list_repr(node[0], rep, level+1)
        list_repr(self._head, rep, 1)
        return 'LinkedList(%s): %s' % (self._size, rep)
    
    def __str__(self):                
        def list_str(node):
            if node is None:                                
                return ''
            val = str(node.value) if type(node.value) != str else "'%s'" % node.value
            return "%s"% val + ', ' + list_str(node[0])
        return 'LinkedList: [' + list_str(self._head)[:-2] + ']'


class LinkedListFast(LinkedList):
    # using parent sttribute in BaseNode class
    def __init__(self, fixed_dtype=True):
        super().__init__(fixed_dtype)
        self._tail = None
    
    def append(self, value):
        self._size += 1
        if self._head is None:
            self._head = self._tail = self._LinkedList__NodeClass(value, 1)
            return
        self._tail[0] = self._LinkedList__NodeClass(value, 1)
        self._tail = self._tail[0]
        
    def pop(self):
        assert self._head is not None, 'Popping from empty list'
        
        self._size -= 1        
        if self._head == self._tail:
            value = self._head.value
            self._head = self._head = None
            return value
        
        value = self._tail.value
        self._tail = self._tail._parent
        return value

In [6]:
# examples
ll = LinkedListFast()
ll.append('a')
ll.append('b')
ll.append('c')
ll.append('d')
ll[2] = 'ccc'
print(ll)
print(repr(ll))
print('*'*30)

ll.pop()
print(ll)
print(['a', 'b', 'ccc'])

LinkedList: ['a', 'b', 'ccc', 'd']
LinkedList(4): {1: {'child_count': 1, 'max_children': 1, 'value': 'a', 'dtype': <class 'str'>}, 2: {'child_count': 1, 'max_children': 1, 'value': 'b', 'dtype': <class 'str'>}, 3: {'child_count': 1, 'max_children': 1, 'value': 'ccc', 'dtype': <class 'str'>}, 4: {'child_count': 0, 'max_children': 1, 'value': 'd', 'dtype': <class 'str'>}}
******************************
LinkedList: ['a', 'b', 'ccc', 'd']
['a', 'b', 'ccc']


# Stack

In [7]:
class Stack:
    def __init__(self):        
        self.__NodeClass = BaseNode
        self._top = None
        self._size = 0        
    
    def __len__(self):
        return self._size    

    
    def push(self, value):
        new_node = self.__NodeClass(value,1)        
        new_node[0] = self._top
        self._top = new_node
        self._size += 1
    
    def pop(self):
        if len(self) == 0:
            return None        
        val = self._top.value        
        self._top = self._top[0] if len(self) > 1 else None
        self._size -= 1
        return val
    
    def __str__(self):                
        def stack_str(node):
            if node is None:                                
                return ''
            val = str(node.value) if type(node.value) != str else "'%s'" % node.value
            return val + ' > ' +"%s"% stack_str(node[0])
        return 'Stack: [' + stack_str(self._top)[:-3] + ']'

In [8]:
# example
s = Stack()
s.push(n)
s.push(root)
s.push(ll)
print(s)
print(len(s), s.pop())
print(len(s), s.pop())
print(len(s))
print(s)

Stack: [LinkedList: ['a', 'b', 'ccc', 'd'] > (ROOT; (l; (ll; _, _), (lr; _, _)), (r; _, (rr; _, (rrr; _, _)))) > (None; l, r)]
3 LinkedList: ['a', 'b', 'ccc', 'd']
2 (ROOT; (l; (ll; _, _), (lr; _, _)), (r; _, (rr; _, (rrr; _, _))))
1
Stack: [(None; l, r)]


# Queue

In [9]:
class Queue:
    def __init__(self, fixed_dtype=True):
        self.__NodeClass = Node if fixed_dtype else BaseNode
        self._first = None
        self._last = None
        self._size = 0        
    
    def __len__(self):
        return self._size        
    
    def push(self, value):
        self._size += 1
        if self._first is None:
            self._first = self._last = self.__NodeClass(value, 1)
            return
        
        self._last[0] = self.__NodeClass(value, 1)
        self._last = self._last[0]        
        
    def pop(self):
        if self._first is None:
            return None
        self._size -= 1
        value = self._first.value
        if self._last == self._first:
            self._first = self._last = None
            return value
        self._first = self._first[0]
        return value
    
    def __repr__(self):
        rep = {}
        def queue_repr(node, rep, level):
            if node is None:                                
                return
            rep[level] = node
            queue_repr(node[0], rep, level+1)
        queue_repr(self._first, rep, 1)
        return 'Queue(%s): %s' % (self._size, rep)
    
    def __str__(self):                
        def queue_str(node):
            if node is None:                                
                return ''
            val = str(node.value) if type(node.value) != str else "'%s'" % node.value
            return "%s"% val + ' > ' + queue_str(node[0])
        return 'Queue: [' + queue_str(self._first)[:-3] + ']'
    

In [10]:
q = Queue()
q.push(n)
q.push(root)
print(q)
print(len(q), q.pop())
print(len(q), q.pop())
print(len(q))
print(q)

Queue: [(None; l, r) > (ROOT; (l; (ll; _, _), (lr; _, _)), (r; _, (rr; _, (rrr; _, _))))]
2 (None; l, r)
1 (ROOT; (l; (ll; _, _), (lr; _, _)), (r; _, (rr; _, (rrr; _, _))))
0
Queue: []


# Tips

## Decorators

In [11]:
# Thanks to https://www.programiz.com/python-programming/decorator

def star(func):
    def inner(*args, **kwargs):
        print('*' * 30)
        func(*args, **kwargs)
        print('*' * 30)
        return
    return inner

def nonnegative_integer(func):
    def inner(x):
        if not isinstance(x, int):
            raise TypeError('factorial is not defined for {}'.format(x))
        if x < 0:
            raise ValueError('factorial is not defined for negative numbers')
        return func(x)
    return inner

def printer(func):
    def inner(*args, **kwargs):
        print(func(*args, **kwargs))
    return inner
        

@nonnegative_integer
def factorial(n:int):
    if n==0 or n==1:
        return 1
    return n * factorial(n-1)

@star
@printer
@nonnegative_integer
def fact(n:int):
    if n==0 or n==1:
        return 1
    res=1
    for i in range(1,n+1):
        res *= i
    return res


def f(n:int):
    if n==0 or n==1:
        return 1
    res=1
    for i in range(1,n+1):
        res *= i
    return res
nondecorated_fact = star(printer(nonnegative_integer(f)))

print(factorial(5))
fact(5)
nondecorated_fact(5)

120
******************************
120
******************************
******************************
120
******************************
